<a href="https://colab.research.google.com/github/adammoss/MLiS2/blob/master/workshops/workshop_llm/transformer_finetune_workshop_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As an example application of a transformer, let's build a GPT like language model that predicts the probability of a sentence of $\tau$ tokens,

$
P \left( \boldsymbol{x}^{(1)}, \boldsymbol{x}^{(2)}, \ldots, \boldsymbol{x}^{(\tau)}    \right) = \prod_{t  = 1}^{\tau} P \left( \boldsymbol{x}^{(i)} |  \boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(i-1)}    \right)
$

where $\boldsymbol{x}^{(t)}$ is a vector representing a token.

In this example we fine-tune GPT to act as a question-answering system.

In [1]:
!pip install tiktoken
!pip install keras_nlp
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.8 MB/s eta 0:00:00


In [2]:
import itertools
import operator
import numpy as np
import sys
from datetime import datetime
import os
import requests
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import time

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

import tensorflow_datasets as tfds

import tiktoken

import keras_nlp
import keras

import tensorflow_text as tf_text

TensorFlow version: 2.15.0
Using TensorFlow backend


In [4]:
batch_size = 16
context_size = 256

In [5]:
train_examples, val_examples = tfds.load(
    'web_questions',
    split=['train', 'test'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3778 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/1.0.0.incompleteNNBZD0/web_questions-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/2032 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/1.0.0.incompleteNNBZD0/web_questions-test.tfrecord*...:   0%…

Dataset web_questions downloaded and prepared to /root/tensorflow_datasets/web_questions/1.0.0. Subsequent calls will reuse this data.


In [6]:
def deconstruct(x):
  return x['question'], x['answers'][0]

In [7]:
@tf.py_function(Tout=tf.string)
def join_input(x, y):
  return '### Question: %s\n### Answer: %s' % (x.numpy().decode('utf-8'), y.numpy().decode('utf-8'))

In [8]:
train_ds = train_examples.map(deconstruct).map(join_input)
test_ds = val_examples.map(deconstruct).map(join_input)

train_ds = train_ds.take(10000)
test_ds = test_ds.take(10000)

BUFFER_SIZE = 10000

train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

test_ds = (
    test_ds
    .batch(batch_size)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [9]:
x = next(iter(train_ds))
print(x.numpy()[0])

b'### Question: how much does the average american pay in taxes over their lifetime?\n### Answer: American National Biography'


In [10]:
x = next(iter(test_ds))
print(x.numpy()[0])

b'### Question: what year did the seahawks win the superbowl?\n### Answer: 2006 NFL season'


In [11]:
# To speed up training and generation, we do not use a the full GPT2 context length of 1024.
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=context_size,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 431kB/s]
100%|██████████| 446k/446k [00:01<00:00, 438kB/s]
100%|██████████| 0.99M/0.99M [00:01<00:00, 798kB/s]
100%|██████████| 484/484 [00:00<00:00, 773kB/s]
100%|██████████| 475M/475M [00:30<00:00, 16.3MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:46: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


In [12]:
start = time.time()

conversation = "### Question: Who is the current president of the United States?\n### Answer:"

output = gpt2_lm.generate(conversation, max_length=200)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")

print(output)

TOTAL TIME ELAPSED: 17.89s
### Question: Who is the current president of the United States?
### Answer: President Barack Obama.
### Question: What is his name?
### Answer: President Obama.
### Question: Who is his wife?
### Answer: President Obama.
### Question: What is her address?
### Answer: President Obama.
### Question: What is the current status of her marriage?
### Answer: President Obama.
### Question: What are her children?
#### Answer: President Obama.
# Question: Who is the current leader of the Democratic Party?
### Answer: President Obama.
### Question: What is the current status of the American Revolution?
# Answer: President Obama.
### Question: What is the current status of the American Republic?
### Answer: President Obama.
# Question: What is the current status of the American Republic of China?
### Answer: President Obama.
# Question: What is the current


In [13]:
gpt2_lm.summary()

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                  ┃ Output Shape                           ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)                     │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ token_ids (InputLayer)                        │ (None, None)                           │               0 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ gpt2_backbone (GPT2Backbone)                  │ (None, None, 768)                      │     124,439,808 │
├───────────────────────────────────────────────┼────────────────────────────────────────┼─────────────────┤
│ token_embedding (ReversibleEmbedding)         │ (None, None, 50257)                    │      38,597,376 │
└───────────────────────────────────────────────┴────────────────────────────────────────┴─────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
num_epochs = 5

# Linearly decaying learning rate.
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=train_ds.cardinality() * num_epochs,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

gpt2_lm.fit(train_ds, epochs=num_epochs, validation_data=test_ds)

Epoch 1/5
236/236 [==============================] - 303s 1s/step - loss: 0.1938 - accuracy: 0.5920 - val_loss: 0.1621 - val_accuracy: 0.6420
Epoch 2/5
236/236 [==============================] - 251s 1s/step - loss: 0.1539 - accuracy: 0.6432 - val_loss: 0.1556 - val_accuracy: 0.6540
Epoch 3/5
236/236 [==============================] - 251s 1s/step - loss: 0.1375 - accuracy: 0.6702 - val_loss: 0.1538 - val_accuracy: 0.6611
Epoch 4/5
236/236 [==============================] - 250s 1s/step - loss: 0.1275 - accuracy: 0.6863 - val_loss: 0.1536 - val_accuracy: 0.6635
Epoch 5/5
236/236 [==============================] - 250s 1s/step - loss: 0.1220 - accuracy: 0.6950 - val_loss: 0.1539 - val_accuracy: 0.6646


In [17]:
start = time.time()

conversation = "### Question: Who is the current president of the United States?\n### Answer:"

output = gpt2_lm.generate(conversation, max_length=200)

end = time.time()
print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")

print(output)

TOTAL TIME ELAPSED: 0.23s
### Question: Who is the current president of the United States?
### Answer: Barack Obama
